<a href="https://www.bigdatauniversity.com"><img src = "https://ibm.box.com/shared/static/cw2c7r3o20w9zn8gkecaeyjhgw3xdgbj.png" width = 400, align = "center"></a>

<h1 align=center><font size = 5>CONTENT-BASED FILTERING</font></h1>

Los sistemas de recomendación son una colección de algoritmos utilizados para recomendar elementos a los usuarios en función de la información tomada del usuario. Estos sistemas se han vuelto omnipresentes y se pueden ver comúnmente en tiendas en línea, bases de datos de películas y buscadores de trabajo. En este cuaderno, exploraremos sistemas de recomendación basados en contenido e implementaremos una versión simple de uno usando Python y la biblioteca Pandas.

### Table of contents

<div class="alert alert-block alert-info" style="margin-top: 20px">
- <p><a href="#ref1">Acquiring the Data</a></p>
- <p><a href="#ref2">Preprocessing</a></p>
- <p><a href="#ref3">Content-Based Filtering</a></p>
<p></p>
</div>
<br>

<a id="ref1"></a>
# Acquiring the Data

Para adquirir y extraer los datos, simplemente ejecute los siguientes scripts de Bash:
Conjunto de datos adquirido de [GroupLens](http://grouplens.org/datasets/movielens/).  Permite descargar el conjunto de datos. Para descargar los datos, usaremos! Wget. Para descargar los datos, usaremos! Wget para descargarlos de IBM Object Storage.
¿Sabías? Cuando se trata de Machine Learning, es probable que trabajes con grandes conjuntos de datos. Como empresa, ¿dónde puede alojar sus datos? IBM ofrece una oportunidad única para las empresas, con 10 Tb de IBM Cloud Object Storage: [Regístrese ahora gratis](http://cocl.us/ML0101EN-IBM-Offer-CC)

In [1]:
!wget -O moviedataset.zip https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/moviedataset.zip
print('unziping ...')
!unzip -o -j moviedataset.zip 

unziping ...


"wget" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.
"unzip" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


¡Ahora está listo para comenzar a trabajar con los datos!

<a id="ref2"></a>
# Preprocesamiento

Primero, eliminemos todas las importaciones:

In [2]:
# Biblioteca de manipulación de marcos de datos
import pandas as pd
# Funciones matemáticas, solo necesitaremos la función sqrt, así que importaremos solo eso
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Ahora leamos cada archivo en sus marcos de datos:

In [3]:
# Almacenamiento de la información de la película en un marco de datos de pandas
movies_df = pd.read_csv('ml-latest/movies.csv')
# Almacenamiento de la información del usuario en un marco de datos de pandas
ratings_df = pd.read_csv('ml-latest/ratings.csv')
#Head es una función que obtiene las primeras N filas de un marco de datos. El valor predeterminado de N es 5.
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


También eliminemos el año de la columna __título__ usando la función de reemplazo de pandas y almacénelo en una nueva columna __año__.

In [4]:
# Uso de expresiones regulares para encontrar un año almacenado entre paréntesis
# Especificamos los paréntesis para no entrar en conflicto con las películas que tienen años en sus títulos
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
#Eliminar los paréntesis
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)
# Eliminar los años de la columna 'título'
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')
# Aplicación de la función de tira para deshacerse de los caracteres de espacios en blanco finales que puedan haber aparecido
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


Con eso, también dividamos los valores en la columna __Genres__ en una __lista de Géneros__ para simplificar el uso futuro. Esto se puede lograr aplicando la función de cadena dividida de Python en la columna correcta.

In [5]:
# Cada género está separado por un | así que simplemente tenemos que llamar a la función de división en |
movies_df['genres'] = movies_df.genres.str.split('|')
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995


Dado que mantener los géneros en un formato de lista no es óptimo para la técnica del sistema de recomendación basada en el contenido, utilizaremos la técnica de codificación en caliente única para convertir la lista de géneros en un vector donde cada columna corresponde a un posible valor de la característica. Esta codificación es necesaria para alimentar datos categóricos. En este caso, almacenamos cada género diferente en columnas que contienen 1 o 0. 1 muestra que una película tiene ese género y 0 muestra que no. También almacenemos este marco de datos en otra variable, ya que los géneros no serán importantes para nuestro primer sistema de recomendaciones.

In [6]:
# Copiar el marco de datos de la película en uno nuevo ya que no necesitaremos usar la información de género en nuestro primer caso.
moviesWithGenres_df = movies_df.copy()

# Por cada fila en el marco de datos, recorra la lista de géneros y coloque un 1 en la columna correspondiente
for index, row in movies_df.iterrows():
    for genre in row['genres']:
        moviesWithGenres_df.at[index, genre] = 1
# Rellenar los valores NaN con 0 para mostrar que una película no tiene el género de esa columna
moviesWithGenres_df = moviesWithGenres_df.fillna(0)
moviesWithGenres_df.head()

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men,"[Comedy, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II,[Comedy],1995,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


A continuación, veamos el marco de datos de calificaciones.

In [7]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,169,2.5,1204927694
1,1,2471,3.0,1204927438
2,1,48516,5.0,1204927435
3,2,2571,3.5,1436165433
4,2,109487,4.0,1436165496


Cada fila en el marco de datos de calificaciones tiene una identificación de usuario asociada con al menos una película, una calificación y una marca de tiempo que muestra cuándo la revisaron. No necesitaremos la columna de marca de tiempo, así que eliminémosla para guardarla en la memoria.

In [8]:
#Drop elimina una fila o columna especificada de un marco de datos
ratings_df = ratings_df.drop('timestamp', 1)
ratings_df.head()

,userId,movieId,rating
0,1,169,2.5
1,1,2471,3.0
2,1,48516,5.0
3,2,2571,3.5
4,2,109487,4.0


<a id="ref3"></a>
# Sistema de recomendación basado en contenido

Ahora, echemos un vistazo a cómo implementar __Sistemas de recomendación basados en contenido__ o __Item-Item__. Esta técnica intenta descubrir cuáles son los aspectos favoritos de un usuario de un elemento, y luego recomienda elementos que presentan esos aspectos. En nuestro caso, vamos a tratar de descubrir los géneros favoritos de la entrada de las películas y calificaciones otorgadas.

Comencemos creando un usuario de entrada para recomendar películas para:

Aviso: para agregar más películas, simplemente aumente la cantidad de elementos en __userInput__. ¡Siéntase libre de agregar más! Solo asegúrese de escribirlo con letras mayúsculas y si una película comienza con "The", como "The Matrix", escríbala así: 'Matrix, The'.

In [9]:
userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5}
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies

,title,rating
0,"Breakfast Club, The",5.0
1,Toy Story,3.5
2,Jumanji,2.0
3,Pulp Fiction,5.0
4,Akira,4.5


#### Añadir movieId al usuario de entrada
Una vez completada la entrada, extraigamos las ID de las películas de entrada del marco de datos de las películas y las agreguemos.

Podemos lograr esto filtrando primero las filas que contienen el título de las películas de entrada y luego fusionando este subconjunto con el marco de datos de entrada. También soltamos columnas innecesarias para la entrada para ahorrar espacio en la memoria.

In [ ]:
# Filtrando las películas por título
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]
# Luego fusionarlo para que podamos obtener el movieId. Se está fusionando implícitamente por título.
inputMovies = pd.merge(inputId, inputMovies)
# Información de caída que no utilizaremos del marco de datos de entrada
inputMovies = inputMovies.drop('genres', 1).drop('year', 1)
# Marco de datos de entrada final
# Si una película que agregaste arriba no está aquí, entonces podría no estar en el original
#dataframe o podría deletrearse de manera diferente, compruebe las mayúsculas.
inputMovies

Comenzaremos por aprender las preferencias de la entrada, así que obtengamos el subconjunto de películas que la entrada ha visto desde el marco de datos que contiene géneros definidos con valores binarios.

In [ ]:
# Filtrando las películas desde la entrada
userMovies = moviesWithGenres_df[moviesWithGenres_df['movieId'].isin(inputMovies['movieId'].tolist())]
userMovies

Solo necesitaremos la tabla de género real, así que limpiemos esto un poco restableciendo el índice y soltando las columnas movieId, title, genres y year.

In [ ]:
# Restablecer el índice para evitar problemas futuros
userMovies = userMovies.reset_index(drop=True)
# Caída de problemas innecesarios debido al ahorro de memoria y para evitar problemas
userGenreTable = userMovies.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
userGenreTable

¡Ahora estamos listos para comenzar a aprender las preferencias de entrada!

Para hacer esto, vamos a convertir cada género en pesos. Podemos hacer esto usando las revisiones de la entrada y multiplicándolas en la tabla de género de la entrada y luego resumiendo la tabla resultante por columna. Esta operación es en realidad un producto de punto entre una matriz y un vector, por lo que simplemente podemos lograrlo llamando a la función "punto" de Pandas.

In [ ]:
inputMovies['rating']

In [ ]:
# Producto de punto para obtener pesos
userProfile = userGenreTable.transpose().dot(inputMovies['rating'])
#El perfil de usuario
userProfile

Ahora, tenemos los pesos para cada una de las preferencias del usuario. Esto se conoce como el perfil de usuario. Con esto, podemos recomendar películas que satisfagan las preferencias del usuario.

Comencemos extrayendo la tabla de géneros del original dataframe:

In [ ]:
# Ahora obtengamos los géneros de cada película en nuestro marco de datos original
genreTable = moviesWithGenres_df.set_index(moviesWithGenres_df['movieId'])
# Y descartar la información innecesaria
genreTable = genreTable.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
genreTable.head()

In [ ]:
genreTable.shape

Con el perfil de entrada y la lista completa de películas y sus géneros en la mano, tomaremos el promedio ponderado de cada película en función del perfil de entrada y recomendaremos las veinte mejores películas que más lo satisfagan.

In [ ]:
# Multiplica los géneros por los pesos y luego toma el promedio ponderado
recommendationTable_df = ((genreTable*userProfile).sum(axis=1))/(userProfile.sum())
recommendationTable_df.head()

In [ ]:
# Ordene nuestras recomendaciones en orden descendente
recommendationTable_df = recommendationTable_df.sort_values(ascending=False)
# Solo un vistazo a los valores
recommendationTable_df.head()

Ahora aquí está la tabla de recomendaciones!

In [ ]:
# La tabla de recomendaciones final
movies_df.loc[movies_df['movieId'].isin(recommendationTable_df.head(20).keys())]

### Ventajas y desventajas del filtrado basado en contenido

##### Ventajas
* Aprende las preferencias del usuario
* Altamente personalizado para el usuario

##### Desventajas
* No tiene en cuenta lo que otros piensan del artículo, por lo que pueden ocurrir recomendaciones de artículos de baja calidad.
* La extracción de datos no siempre es intuitiva
* No siempre es obvio determinar qué características del elemento no le gustan o le gustan al usuario

## Want to learn more?

IBM SPSS Modeler is a comprehensive analytics platform that has many machine learning algorithms. It has been designed to bring predictive intelligence to decisions made by individuals, by groups, by systems – by your enterprise as a whole. A free trial is available through this course, available here: [SPSS Modeler](http://cocl.us/ML0101EN-SPSSModeler).

Also, you can use Watson Studio to run these notebooks faster with bigger datasets. Watson Studio is IBM's leading cloud solution for data scientists, built by data scientists. With Jupyter notebooks, RStudio, Apache Spark and popular libraries pre-packaged in the cloud, Watson Studio enables data scientists to collaborate on their projects without having to install anything. Join the fast-growing community of Watson Studio users today with a free account at [Watson Studio](https://cocl.us/ML0101EN_DSX)

### Thanks for completing this lesson!

Notebook created by: <a href = "https://ca.linkedin.com/in/saeedaghabozorgi">Saeed Aghabozorgi</a>, Gabriel Garcez Barros Sousa

<hr>
Copyright &copy; 2018 [Cognitive Class](https://cocl.us/DX0108EN_CC). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).​